In [1]:
import sys,os 
sys.path.append('/home/benr/ACT/CW2/py')
import numpy as np 


import pandas as pd 
from PIL import Image, UnidentifiedImageError
from sklearn.preprocessing import LabelEncoder
from functions import get_data,split_data_torch,train_q2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

2025-12-03 15:53:47.671075: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-03 15:53:47.722701: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-03 15:53:48.982953: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

CUDA available: True
CUDA version: 12.8
CUDA_VISIBLE_DEVICES: None


# Question 2, Convolutional Neural Network for image recognition 
In this section the aim is to repeat the same process as question 1 but replace the traditional reandom forest method with a convolutional neural network. The expectation is that due to the CNN's ability to correlate spacial features, it will be a more roubust method of recognising galaxy morphology.

The first step will be to load the data in exactly the same way as Q1. 

In [3]:

# get the data from functions.py
images,labels = get_data()
# convert images to pytorch tnesors
images = torch.from_numpy(images)
labels = torch.from_numpy(labels).long()

print(images.shape)


/home/benr/.astroNN/datasets/Galaxy10_DECals.h5 was found!
torch.Size([17736, 256, 256, 3])


In [4]:
# Move channels: (N,H,W,C) -> (N,C,H,W)
images = images.permute(0, 3, 1, 2)      

# standardise 
images = images.float() / 255.0      
mean = images.mean(dim=[0,2,3])
std = images.std(dim=[0,2,3])
print(images.shape)
print(mean)
print(std)

torch.Size([17736, 3, 256, 256])
tensor([0.1675, 0.1626, 0.1589])
tensor([0.1287, 0.1180, 0.1116])


For the CNN the lables will need to be encoded into numerical values. 
Documentation can be found here: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

Now we set up the class that will hold the structure of the network. Galaxy_CNN takes the number of input channels, the number of classes (Prediction catagories) and passes them two the different layers of the network. 
Throughout the process of data being passed through different levels of the CNN we will need to keep trach of the network is change the size of each tensor. This can be done with the following formulas

* CNN Output - $$\frac{H + 2P -K}{s} + 1 $$
* Pooling Output - $$\frac{H-K}{s}  + 1$$

H = input height of width of image 


K = kernel size

P = padding size

s = stride 

In [5]:

class Galaxy_CNN(nn.Module):
    def __init__(self, inCH, nCL):  
        super(Galaxy_CNN, self).__init__()
        # conv blocks
        self.conv1 = nn.Conv2d(3, inCH, kernel_size=7, stride=1, padding=4)
        self.bn1   = nn.BatchNorm2d(inCH)

        self.conv2 = nn.Conv2d(inCH, inCH*2, kernel_size=5, stride=2, padding=1)
        self.bn2   = nn.BatchNorm2d(inCH*2)

        self.conv3 = nn.Conv2d(inCH*2, inCH, kernel_size=3, stride=1, padding=1)
        self.bn3   = nn.BatchNorm2d(inCH)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.4)

        self.lin = nn.Linear(inCH*16**2, nCL)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout(x)
            
        x = x.view(x.size(0), -1)    
        x = self.lin(x)
        return x

In [6]:
#split tensor into training and test sets (see functions.py)
I = np.arange(len(labels))
xtrn,xtst,ytrn,ytst = split_data_torch(images,labels,I)



train_ds = TensorDataset(xtrn, ytrn)
test_ds  = TensorDataset(xtst, ytst)




train_loader = DataLoader(
    train_ds, batch_size=5, shuffle=True,
    num_workers=4, pin_memory=True
)

test_loader  = DataLoader(
    test_ds, batch_size=5, shuffle=False,
    num_workers=4, pin_memory=True
)


In [7]:
num_classes = len(labels.unique())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Galaxy_CNN(128,num_classes).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=10**-3,weight_decay=1e-4)

#for normalisation 
mean_tensor = mean.view(1,3,1,1).to(device)
std_tensor = std.view(1,3,1,1).to(device)


print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))


0
NVIDIA GeForce RTX 4060 Laptop GPU


In [8]:
train_q2(model, train_loader,test_loader,  
      mean_tensor,std_tensor,
      criterion, 
      optimizer, device, 
      n_epoch=70)


Saved new best model (acc=0.4476)
Epoch 1/70 Train loss: 2.1748  Train acc: 0.340  Test acc: 0.448
Saved new best model (acc=0.4991)
Epoch 2/70 Train loss: 1.4875  Train acc: 0.463  Test acc: 0.499
Saved new best model (acc=0.5160)
Epoch 3/70 Train loss: 1.3186  Train acc: 0.529  Test acc: 0.516
Saved new best model (acc=0.5720)
Epoch 4/70 Train loss: 1.2176  Train acc: 0.567  Test acc: 0.572
Epoch 5/70 Train loss: 1.1032  Train acc: 0.608  Test acc: 0.572
Epoch 6/70 Train loss: 1.0022  Train acc: 0.643  Test acc: 0.572
Saved new best model (acc=0.5840)
Epoch 7/70 Train loss: 0.9197  Train acc: 0.671  Test acc: 0.584
Saved new best model (acc=0.5994)
Epoch 8/70 Train loss: 0.8355  Train acc: 0.701  Test acc: 0.599
Saved new best model (acc=0.6062)
Epoch 9/70 Train loss: 0.7633  Train acc: 0.732  Test acc: 0.606
Saved new best model (acc=0.6073)
Epoch 10/70 Train loss: 0.7093  Train acc: 0.750  Test acc: 0.607
Epoch 11/70 Train loss: 0.6523  Train acc: 0.773  Test acc: 0.594
Saved new b